<a href="https://colab.research.google.com/github/ersilia-os/event-fund-ai-drug-discovery/blob/main/notebooks/session2_skills.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 2: 
In this session, we will use a simple dataset to train a basic Machine Learning Model and understand the steps involved in their development.

*Disclaimer: this pipeline is an example prepared with curated data and should not be reproduced with the student's own datasets. The goal of this workshop is purely academic and does not represent a real research case study.*

## Datasets

In this exercise, we will leverage the [Therapeutics Data Commons](https://tdcommons.ai/) database, an open science effort to collect AI-ready datasets for biomedical questions (read more on its [publication](https://arxiv.org/abs/2102.09548)).

The data cleaning steps discussed in Session 1 have already been done for the TDC data, so we can focus directly on the generation of ML models. We will retrieve two toxicity-related datasets from TDC to train a classification model (2.1) and a regression model (2.2)

## 2.1 Classification Task
A classification model identifies the "category" of a new datapoint. In most QSAR models, the model learns how to classify molecules in:


*   Active: labelled with 1 (the molecule has an effect against the specific target or pathogen)
*   Inactive: labelled with 0 (the molecule does not have an effect against the specific target or pathogen)


### 2.1.1 Data Preparation

We will model a critical step in the drug discovery development, **cardiotoxicty due to hERG blockade**. hERG is a potassium channel whose blockage causes prolongued QT intervals and eventually cardiotoxicity, and is one of the major adverse drug reactions that cause compound attrition in the drug discovery pipelines. Therefore, it is essential to identify putative hERG blockers early on the drug discovery cascade.

In [ ]:
#we first connect to drive
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Download TDC hERG Dataset
#@markdown Click on the play button to get the train and tests sets for the TDC hERG data
#@markdown By running this cell, you will get the datasets saved to the drive in data/session2

%%capture

#TDC can be installed and accessed as a Python package
!pip install PyTDC

# import the hERG dataset, part of the Toxicity data available at TDC
from tdc.single_pred import Tox
data = Tox(name = "hERG")

split = data.get_split()

#we can now separate the compressed dataset in the three sections
train = split["train"]
validation = split["valid"]
test = split["test"]

train.to_csv("drive/MyDrive/h3d_ersilia_ai_workshop/data/session2/tdc_herg_train.csv", index=False)
validation.to_csv("drive/MyDrive/h3d_ersilia_ai_workshop/data/session2/tdc_herg_validation.csv", index=False)
test.to_csv("drive/MyDrive/h3d_ersilia_ai_workshop/data/session2/tdc_herg_test.csv", index=False)

#### Data Splits
We will start by splitting the data in 3 parts:

*   Train: the portion of the dataset that will be used to train the model
*   Valid: the validation set, which will be used during the modelling to assert and improve the performance of the algorithm
*   Test: a portion of the dataset kept completely separated from the data used to train the model. We will use this to evaluate the final model performance.

Usually, this process is done manually and must ensure that the three datasets are balanced (i.e. have an equal representation of positives and negatives and are representative of all the chemical space). 

For the purpose of the course, we will use the prepared TDC split which already guarantees the above conditions.


In [ ]:
#we can check how many molecules we have in each set
print(len(train))
print(len(validation))
print(len(test))

*In this example, we will only use the train and test sets, since we will not be testing different model parameters*

In [ ]:
#let's explore the data
#by convention, the input (molecules in this case) is named X, and the output (bioactivity) is Y
train.head() 

In [ ]:
#we can check the number of positives and negatives in the train set
print(len(train[train["Y"]==0])) #not cardiotoxic
print(len(train[train["Y"]==1])) #cardiotoxic

#### Data Visualization
Finally, before moving onto model training, we can leverage the Python Package RdKit, the largest open source toolbox for chemioinformatics, to explore a bit more the chemical structures of our data

In [ ]:
%%capture
#we first need to install rdkit in Google Colab and import the packages of interest
!pip install rdkit

from rdkit import Chem
from rdkit.Chem import Draw

In [ ]:
#we select a list of smiles

smiles = train["Drug"][449:] #the last 9 molecules
smiles

In [ ]:
#convert the smiles to RdKit Molecules (not readable by humans)
mols = [Chem.MolFromSmiles(smi) for smi in smiles]
mols[0] #again, we can check what is the "mols" we have created

In [ ]:
#use the Draw function to visualise the molecules
Draw.MolsToGridImage(mols)

### 2.1.2 Molecule Featurization

To train an ML model, we need to be able to pass the molecules to the computer in a format that the computer can understand. That is, numerical vectors or images.

In this case, we will use the Chemical Checker to create signatures encompassing not only structural characteristics but also the bioactivity profile of the molecules.

In [ ]:
%%capture
#first, we install the signaturizer and import it
!pip install signaturizer
from signaturizer import Signaturizer
sign = Signaturizer("GLOBAL")

In [ ]:
#we then convert the smiles into vectors (X)
X_train = sign.predict(train["Drug"]).signature
X_test = sign.predict(test["Drug"]).signature

In [ ]:
# we can see how a molecule in the train set has been converted to a vector
X_train[0] #0 indicates the first molecule in the list

In [ ]:
#finally, we need to prepare the outputs (Y), creating three lists:
Y_train = list(train["Y"])
Y_test = list(test["Y"])

### 2.1.3. Supervised Machine Learning

We will use the SciKit-Learn Python package to train an ML model based on a Random Forest algorithm. In this case, since the data is already binarized (0 and 1 for inactive and active, instead of continuous experimental results like IC50) we will train a Classifier.

In [ ]:
%%capture

#install scikit-learn (sklearn) and import the RandomForest function
!pip install sklearn
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#in ML, the training of a model is called "fitting" the model to the data (the molecules and outputs of the Train set)

clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

#### 2.1.4 Understanding Classification outputs

A Classifier gives back two numbers:


*   Probability of 0 (inactive)
*   Probability of 1 (active)



In [ ]:
y_pred = clf.predict_proba(X_test)
y_pred[:10] #let's check the first 10 results

In [ ]:
y_pred[:,1] #we are interested only in the probability of active (proba1 or second column)

#### 2.1.4 Model Evaluation
To understand whether a model is performing correctly or not, we have several measures. Here, we will use two of them:


*   Confusion matrices: a table that indicates how many molecules were correctly classified by the model and how many were misclassified.
*   ROC Curve: a probability curve showing the True Positive and False Positive Rates at different classification thresholds.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay as cdm

cdm.from_estimator(clf, X_test, Y_test) #we use the test set to check model performance

In [ ]:
from sklearn.metrics import precision_score, recall_score
#precision and recall scores need a specific threshold

y_pred_bin = []
for x in y_pred[:,1]:
    if x > 0.5:
        y_pred_bin += [1]
    if x <= 0.5:
        y_pred_bin += [0]
precision = precision_score(Y_test, y_pred_bin)
recall = recall_score(Y_test, y_pred_bin)
print(precision, recall)

In [ ]:
from sklearn.metrics import RocCurveDisplay as rdc

rdc.from_estimator(clf, X_test, Y_test) #we use the test set to check model performance

In [ ]:
#we can also try to predict the values in the training set and see how good our algorithm does:
cdm.from_estimator(clf, X_train, Y_train)

In [ ]:
rdc.from_estimator(clf, X_train, Y_train)

In [ ]:
from sklearn.metrics import precision_score, recall_score
#precision and recall scores need a specific threshold

train_pred = clf.predict_proba(X_train)[:,1]
y_pred_bin = []
for x in train_pred:
    if x > 0.5:
        y_pred_bin += [1]
    if x <= 0.5:
        y_pred_bin += [0]
precision = precision_score(Y_train, y_pred_bin)
recall = recall_score(Y_train, y_pred_bin)
print(precision, recall)

## 2.2 Regression Task
Regression models predict continuous numerical values. Applied to the biomedical field, a regression model might predict the IC50 against a pathogen, or the % of inhibition of a specific target. 

### 2.2.1 Data Preparation
We will use the Acute Toxicity LD50 dataset from TDCommons. It contains over 7000 molecules and its associated Lethal Dose 50 (the amount of an ingested substance in mg/kg that kills 50 percent of a test sample)

In [ ]:
#@title Download TDC LD50 Dataset
#@markdown Click on the play button to get the train and tests sets for the TDC hERG data
#@markdown By running this cell, you will get the datasets saved to the drive in data/session2

%%capture

#TDC can be installed and accessed as a Python package
!pip install PyTDC

# import the LD50 dataset, part of the Toxicity data available at TDC
from tdc.single_pred import Tox
data = Tox(name = 'LD50_Zhu')

split = data.get_split()

#we can now separate the compressed dataset in the three sections
train = split["train"]
validation = split["valid"]
test = split["test"]

train.to_csv("drive/MyDrive/h3d_ersilia_ai_workshop/data/session2/tdc_ld50_train.csv", index=False)
validation.to_csv("drive/MyDrive/h3d_ersilia_ai_workshop/data/session2/tdc_ld50_validation.csv", index=False)
test.to_csv("drive/MyDrive/h3d_ersilia_ai_workshop/data/session2/tdc_ld50_test.csv", index=False)

In [ ]:
#we can check how many molecules we have in each set
print(len(train))
print(len(validation))
print(len(test))

*Again, we will only be using the train and test sets for this example*

In [ ]:
#explore the data
train.head()

In [ ]:
# in this case, we can visualise the data not as active / inactive but as a distribution.

import matplotlib.pyplot as plt

plt.hist(train["Y"], bins=50, color = "#50285a")
plt.xlabel("Lethal Dose 50")
plt.ylabel("Number of molecules")

### 2.2.2 Featurization

In [ ]:
%%capture
#first, we install the signaturizer and import it if we are running the notebook again, else it is not necessary
!pip install signaturizer
from signaturizer import Signaturizer
sign = Signaturizer("GLOBAL")

In [ ]:
#we then convert the smiles (X)
X_train = sign.predict(train["Drug"]).signature
X_test = sign.predict(test["Drug"]).signature

In [ ]:
X_train[0]

In [ ]:
#finally, we need to prepare the outputs (Y), creating three lists:
Y_train = list(train["Y"])
Y_test = list(test["Y"])

In [ ]:
Y_train[0]

### 2.2.3 Model Training
We will use the simplest regression algorithm, a Linear Regression, and evaluate how well it performs with our data

In [ ]:
%%capture
#import the Linear Regression function
from sklearn.linear_model import LinearRegression

In [ ]:
#in ML, the training of a model is called "fitting" the model to the data (the molecules and outputs of the Train set)
reg = LinearRegression()
reg.fit(X_train, Y_train)

In [ ]:
#we can now predict the outcome of the test set
y_pred = reg.predict(X_test)

In [ ]:
#if we check, the prediction is simply a list of LD50
y_pred[:4]

### 2.2.4 Model Evaluation
The best way to understand if the model predictions are accurate is to plot a scatter distribution of real vs predicted values

In [ ]:
import numpy as np

fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.scatter(x=y_pred, y=Y_test, color="#dca0dc")
ax.set_xlabel("Predicted LD50")
ax.set_ylabel("Real LD50")
#for better visualisation, we also plot a diagonal line
ax.plot([np.array(Y_test).min(), np.array(Y_test).max()], [np.array(Y_test).min(), np.array(Y_test).max()], 'r--', lw=3, color = "#50285a") 


In additon, we can use the following metrics:
*   Mean Absolute Error (MAE)
*   Mean Squared Error (MSE)
*   R2 score





In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(Y_test, y_pred)
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

print("MAE: {}".format(mae))
print("MSE: {}".format(mse))
print("R2: {}".format(r2))

### 2.2.5 Improving model performance
We have used the simplest model, a linear regression, which is unable to produce accurate predictions for the LD50.
We can try with a more advanced ML algorithm, like a RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators=10) #we use only 10 trees for speed time
reg.fit(X_train, Y_train)

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.scatter(x=y_pred, y=Y_test, color="#dca0dc")
ax.set_xlabel("Predicted LD50")
ax.set_ylabel("Real LD50")
ax.plot([np.array(Y_test).min(), np.array(Y_test).max()], [np.array(Y_test).min(), np.array(Y_test).max()], 'r--', lw=3, color = "#50285a")

In [ ]:
mae = mean_absolute_error(Y_test, y_pred)
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

print("MAE: {}".format(mae))
print("MSE: {}".format(mse))
print("R2: {}".format(r2))

# Conclusions:

We have learnt the basic steps to train a classification and a regression machine learning model based on bioactivity data. If this was a real-case scenario we would now use the validation results to improve the parameters on the algorithms and obtain more accurate predictions. Another avenue to improve predictions could also be testing different featurization steps for the molecules.
